# FM 24 Moneyball

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import re

# Import created functions
import sys
sys.path.append(r'C:\Users\kevmi\Documents\FM 24 Moneyball\FM-24-Moneyball')

from money_ball_util import *

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [ ]:
### Read raw data

# Select file path for raw data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM 24 exports\Summer 2024\Strikers.html"

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM Data Lab outputs\Winter 2023\Strikers.csv"

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [29]:
df_players.head()

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals/90,minutes/goal,xg/90,xg/shot,xg_overperformance/90,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance/90,goals_outside_box/90,assists/90,minutes/assist,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions/90,xgc/90,xgc_overperformance/90,non_pen_goal_contributions/90,non_pen_xgc/90,non_pen_xgc_overperformance/90,conversion_%,shots/90,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister
0,168222,Mats Seuntjens,"M (LC), AM (RLC), ST (C)",31,187 cm,78 kg,Unh,RKC Waalwijk,Eredivisie,NED,-,-,Scouting Required,"$3,200 p/w",$160K - $1.6M,-,Right,15,1305,6.60,0,87.00,0,0.07,1305.00,0.07,0.06,0.00,0.07,0.07,1305.00,0.07,0.00,0.06,0.00,0.00,0.14,652.50,0.08,0.79,0.05,0.00,0.21,0.15,0.06,0.21,0.15,0.06,7,1.03,0.48,47,0.48,31.45,25.45,81,1.03,3.29,0.90,2.85,0.62,1.97,0.07,0.07,9.38,1.59,16.91,5.38,11.52,-6.14,-53.30,2.00,8.69,4.00,46.03,4.69,53.97,0.00,8.69,52.20,0,0,12.07,2.52,1.03,8.99,0,4.55,18,100,65,57,51,7,5,20,25,16,13,6,51,59,54
1,211522,Chris Bedia,ST (C),27,190 cm,85 kg,-,Union Berlin,Bundesliga,CIV (FRA),-,-,Scouting Required,"$24,500 p/w",$450K - $4.4M,-,Left,11,1095,6.73,7,60.83,1,0.16,547.50,0.26,0.14,-0.10,0.16,0.09,547.50,0.26,-0.10,0.14,-0.10,0.00,0.08,1095.00,0.04,0.51,0.04,0.00,0.25,0.30,-0.06,0.25,0.30,-0.06,9,1.89,0.66,35,0.33,24.49,20.14,82,0.49,2.01,0.49,2.01,0.41,1.68,0.50,0.08,7.23,1.15,15.91,5.42,8.96,-3.54,-39.51,1.32,12.49,6.00,48.03,6.49,51.97,0.08,12.49,50.00,0,0,7.32,2.04,0.58,10.08,0,3.86,4,100,48,11,9,2,1,2,3,6,2,14,82,77,29
2,11022221,Vedat Muriqi,ST (C),29,194 cm,92 kg,-,Mallorca,LaLiga EA Sports,KOS (ALB),-,-,Scouting Required,"$82,000 p/w",$24M - $30M,-,Left,19,1341,6.88,0,70.58,2,0.47,191.57,0.36,0.20,0.11,0.34,0.19,268.20,0.26,0.08,0.14,0.08,0.00,0.00,0.00,0.03,-0.49,-0.03,0.00,0.47,0.39,0.08,0.34,0.29,0.04,26,1.81,0.94,52,0.13,27.85,22.08,79,0.40,1.45,0.40,1.45,0.40,1.45,0.13,0.13,9.33,2.42,25.90,6.04,9.19,-3.15,-34.28,0.74,14.56,7.85,53.92,6.71,46.08,0.07,14.56,54.72,1,0,9.53,2.00,0.54,12.27,0,3.15,5,100,24,0,3,4,11,1,9,12,15,57,80,83,2
3,11024744,Myrto Uzuni,"M (L), AM (RLC), ST (C)",28,178 cm,72 kg,-,Granada,LaLiga EA Sports,ALB,-,-,Scouting Required,"$11,500 p/w",$90K - $8.8M,-,Right,12,996,6.59,2,71.14,3,0.18,498.00,0.16,0.15,0.02,0.18,0.17,498.00,0.16,0.02,0.15,0.02,0.00,0.09,996.00,0.10,-0.06,-0.01,0.00,0.27,0.26,0.01,0.27,0.26,0.01,17,1.08,0.45,42,0.27,33.43,28.73,86,1.72,5.14,0.81,2.43,0.54,1.62,0.18,0.18,15.45,3.07,19.88,7.14,12.29,-5.15,-41.90,2.44,5.96,1.81,30.30,4.16,69.70,0.00,5.96,50.89,0,0,15.72,3.07,1.08,11.11,0,5.60,42,100,94,72,48,22,17,51,30,37,16,17,26,82,43
4,12092528,Datro Fofana,"AM (R), ST (C)",21,181 cm,76 kg,-,Burnley,Premier League,CIV,-,-,Scouting Required,"$69,000 p/w",$21M - $33M,-,Right,13,1187,6.93,6,62.47,4,0.30,296.75,0.3

In [30]:
# # Keep only natural DMs
# path = 'data/Natural DMs.html'

# # Ingest in a dataframe
# df_dms = pd.read_html(path, encoding='utf-8')[0]

# # Format column names 
# df_dms.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_dms.columns]

# df_dms['natural_dm'] = 'Yes'

# df_dms = df_dms[['uid', 'natural_dm']]

# df_players = df_players.merge(df_dms, 
#                               left_on='id',
#                               right_on='uid',
#                               how='left')

# df_players = df_players[df_players['natural_dm'] == 'Yes']

In [ ]:
# Apply to your DataFrame
df_players['price_estimate_M'] = df_players['transfer_value'].apply(parse_transfer_value) / 1000000

## Chance conversion analysis

### Full universe

In [33]:
# Initialize list of targets
targets = df_players['id']

scatter_plot(df_players,
             x_metric='non_pen_xg/90', 
             y_metric='non_pen_goals/90',
             z_metric='non_pen_goals/shot',
             title='Chance conversion', 
             show_name_label=False
)

In [34]:
# Create new metric 
df_players['non_pen_goals/non_pen_xg/90'] = df_players['non_pen_goals/90'] / df_players['non_pen_xg/90'] 

# Filter data 
mask = (
        (df_players['non_pen_xg/90'] >= 0.1) & \
        (df_players['non_pen_goals/90'] >= 0.1) & \
        (df_players['non_pen_goals/shot'] >= 0.04) & \
        (df_players['non_pen_goals/non_pen_xg/90'] >= 0.75) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='non_pen_xg/90', 
             y_metric='non_pen_goals/90',
             z_metric='non_pen_goals/shot',
             title='Chance conversion', 
             show_name_label=False
)

# Keep only players that fulfill criteria
#targets = list(df_targets['id'])

In [35]:
df_players.head(2)

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals/90,minutes/goal,xg/90,xg/shot,xg_overperformance/90,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance/90,goals_outside_box/90,assists/90,minutes/assist,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions/90,xgc/90,xgc_overperformance/90,non_pen_goal_contributions/90,non_pen_xgc/90,non_pen_xgc_overperformance/90,conversion_%,shots/90,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M,non_pen_goals/non_pen_xg/90
0,168222,Mats Seuntjens,"M (LC), AM (RLC), ST (C)",31,187 cm,78 kg,Unh,RKC Waalwijk,Eredivisie,NED,-,-,Scouting Required,"$3,200 p/w",$160K - $1.6M,-,Right,15,1305,6.60,0,87.00,0,0.07,1305.0,0.07,0.06,0.0,0.07,0.07,1305.0,0.07,0.0,0.06,0.0,0.0,0.14,652.5,0.08,0.79,0.05,0.0,0.21,0.15,0.06,0.21,0.15,0.06,7,1.03,0.48,47,0.48,31.45,25.45,81,1.03,3.29,0.90,2.85,0.62,1.97,0.07,0.07,9.38,1.59,16.91,5.38,11.52,-6.14,-53.30,2.00,8.69,4.0,46.03,4.69,53.97,0.00,8.69,52.2,0,0,12.07,2.52,1.03,8.99,0,4.55,18,100,65,57,51,7,5,20,25,16,13,6,51,59,54,1.056,1.000000
1,211522,Chris Bedia,ST (C),27,190 cm,85 kg,-,Union Berlin,Bundesliga,CIV (FRA),-,-,Scouting Required,"$24,500 p/w",$450K - $4.4M,-,Left,11,1095,6.73,7,60.83,1,0.16,547.5,0.26,0.14,-0.1,0.16,0.09,547.5,0.26,-0.1,0.14,-0.1,0.0,0.08,1095.0,0.04,0.51,0.04,0.0,0.25,0.30,-0.06,0.25,0.30,-0.06,9,1.89,0.66,35,0.33,24.49,20.14,82,0.49,2.01,0.49,2.01,0.41,1.68,0.50,0.08,7.23,1.15,15.91,5.42,8.96,-3.54,-39.51,1.32,12.49,6.0,48.03,6.49,51.97,0.08,12.49,50.0,0,0,7.32,2.04,0.58,10.08,0,3.86,4,100,48,11,9,2,1,2,3,6,2,14,82,77,29,2.910,0.615385


## Goal efficiency

In [36]:
# Full universe 
scatter_plot(df_targets,
             x_metric='shots/90', 
             y_metric='conversion_%',
             z_metric='non_pen_xg_overperformance/90',
             title='Efficiency', 
             show_name_label=False
)

In [37]:
# Filter data 
mask = (
        (df_players['shots/90'] >= 1) & \
        (df_players['conversion_%'] >= 14) & \
        (df_players['non_pen_xg_overperformance/90'] >= 0) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='shots/90', 
             y_metric='conversion_%',
             z_metric='non_pen_xg_overperformance/90',
             title='Efficiency', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Creativity

In [38]:
df_players.head()

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals/90,minutes/goal,xg/90,xg/shot,xg_overperformance/90,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance/90,goals_outside_box/90,assists/90,minutes/assist,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions/90,xgc/90,xgc_overperformance/90,non_pen_goal_contributions/90,non_pen_xgc/90,non_pen_xgc_overperformance/90,conversion_%,shots/90,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M,non_pen_goals/non_pen_xg/90
0,168222,Mats Seuntjens,"M (LC), AM (RLC), ST (C)",31,187 cm,78 kg,Unh,RKC Waalwijk,Eredivisie,NED,-,-,Scouting Required,"$3,200 p/w",$160K - $1.6M,-,Right,15,1305,6.60,0,87.00,0,0.07,1305.00,0.07,0.06,0.00,0.07,0.07,1305.00,0.07,0.00,0.06,0.00,0.00,0.14,652.50,0.08,0.79,0.05,0.00,0.21,0.15,0.06,0.21,0.15,0.06,7,1.03,0.48,47,0.48,31.45,25.45,81,1.03,3.29,0.90,2.85,0.62,1.97,0.07,0.07,9.38,1.59,16.91,5.38,11.52,-6.14,-53.30,2.00,8.69,4.00,46.03,4.69,53.97,0.00,8.69,52.20,0,0,12.07,2.52,1.03,8.99,0,4.55,18,100,65,57,51,7,5,20,25,16,13,6,51,59,54,1.056,1.000000
1,211522,Chris Bedia,ST (C),27,190 cm,85 kg,-,Union Berlin,Bundesliga,CIV (FRA),-,-,Scouting Required,"$24,500 p/w",$450K - $4.4M,-,Left,11,1095,6.73,7,60.83,1,0.16,547.50,0.26,0.14,-0.10,0.16,0.09,547.50,0.26,-0.10,0.14,-0.10,0.00,0.08,1095.00,0.04,0.51,0.04,0.00,0.25,0.30,-0.06,0.25,0.30,-0.06,9,1.89,0.66,35,0.33,24.49,20.14,82,0.49,2.01,0.49,2.01,0.41,1.68,0.50,0.08,7.23,1.15,15.91,5.42,8.96,-3.54,-39.51,1.32,12.49,6.00,48.03,6.49,51.97,0.08,12.49,50.00,0,0,7.32,2.04,0.58,10.08,0,3.86,4,100,48,11,9,2,1,2,3,6,2,14,82,77,29,2.910,0.615385
2,11022221,Vedat Muriqi,ST (C),29,194 cm,92 kg,-,Mallorca,LaLiga EA Sports,KOS (ALB),-,-,Scouting Required,"$82,000 p/w",$24M - $30M,-,Left,19,1341,6.88,0,70.58,2,0.47,191.57,0.36,0.20,0.11,0.34,0.19,268.20,0.26,0.08,0.14,0.08,0.00,0.00,0.00,0.03,-0.49,-0.03,0.00,0.47,0.39,0.08,0.34,0.29,0.04,26,1.81,0.94,52,0.13,27.85,22.08,79,0.40,1.45,0.40,1.45,0.40,1.45,0.13,0.13,9.33,2.42,25.90,6.04,9.19,-3.15,-34.28,0.74,14.56,7.85,53.92,6.71,46.08,0.07,14.56,54.72,1,0,9.53,2.00,0.54,12.27,0,3.15,5,100,24,0,3,4,11,1,9,12,15,57,80,83,2,32.400,1.307692
3,11024744,Myrto Uzuni,"M (L), AM (RLC), ST (C)",28,178 cm,72 kg,-,Granada,LaLiga EA Sports,ALB,-,-,Scouting Required,"$11,500 p/w",$90K - $8.8M,-,Right,12,996,6.59,2,71.14,3,0.18,498.00,0.16,0.15,0.02,0.18,0.17,498.00,0.16,0.02,0.15,0.02,0.00,0.09,996.00,0.10,-0.06,-0.01,0.00,0.27,0.26,0.01,0.27,0.26,0.01,17,1.08,0.45,42,0.27,33.43,28.73,86,1.72,5.14,0.81,2.43,0.54,1.62,0.18,0.18,15.45,3.07,19.88,7.14,12.29,-5.15,-41.90,2.44,5.96,1.81,30.30,4.16,69.70,0.00,5.96,50.89,0,0,15.72,3.07,1.08,11.11,0,5.60,42,100,94,72,48,22,17,51,30,37,16,17,26,82,43,5.334,1.125000
4,12092528,Datro Fofana,"AM (R), ST (C)",21,181 cm,76 kg,-,Burnley,Premier

In [39]:
# Full universe 
scatter_plot(df_targets,
             x_metric='open_play_key_passes/90', 
             y_metric='chances_created/90',
             z_metric='open_play_key_pass_%',
             title='Creativity', 
             show_name_label=False
)

In [42]:

# Filter data 
mask = (
        (df_players['open_play_key_passes/90'] >= 0.35) & \
        (df_players['chances_created/90'] >= 0.2) & \
        (df_players['open_play_key_pass_%'] >= 1) & \
        (df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='open_play_key_passes/90', 
             y_metric='chances_created/90',
             z_metric='open_play_key_pass_%',
             title='Creativity', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Pressure quality

In [43]:
# Full universe 
scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

In [53]:
# Filter data 
mask = (
        (df_players['pressures_completed/90'] >= 1) & \
        (df_players['pressure_success_%'] >= 18) & \
        (df_players['duels_win_%'] >= 30) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Presence

In [49]:
df_players.head(1)

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals/90,minutes/goal,xg/90,xg/shot,xg_overperformance/90,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance/90,goals_outside_box/90,assists/90,minutes/assist,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions/90,xgc/90,xgc_overperformance/90,non_pen_goal_contributions/90,non_pen_xgc/90,non_pen_xgc_overperformance/90,conversion_%,shots/90,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M,non_pen_goals/non_pen_xg/90
0,168222,Mats Seuntjens,"M (LC), AM (RLC), ST (C)",31,187 cm,78 kg,Unh,RKC Waalwijk,Eredivisie,NED,-,-,Scouting Required,"$3,200 p/w",$160K - $1.6M,-,Right,15,1305,6.6,0,87.0,0,0.07,1305.0,0.07,0.06,0.0,0.07,0.07,1305.0,0.07,0.0,0.06,0.0,0.0,0.14,652.5,0.08,0.79,0.05,0.0,0.21,0.15,0.06,0.21,0.15,0.06,7,1.03,0.48,47,0.48,31.45,25.45,81,1.03,3.29,0.9,2.85,0.62,1.97,0.07,0.07,9.38,1.59,16.91,5.38,11.52,-6.14,-53.3,2.0,8.69,4.0,46.03,4.69,53.97,0.0,8.69,52.2,0,0,12.07,2.52,1.03,8.99,0,4.55,18,100,65,57,51,7,5,20,25,16,13,6,51,59,54,1.056,1.0


In [51]:
# Full universe 
scatter_plot(df_targets,
             x_metric='attacking_actions/90', 
             y_metric='creative_actions/90',
             z_metric='sprints/90',
             title='Presence', 
             show_name_label=True
) 

In [54]:
# Filter data 
mask = (
        (df_players['attacking_actions/90'] >= 2.2) & \
        (df_players['creative_actions/90'] >= 0.75) & \
        (df_players['sprints/90'] >= 6) & \
        (df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='attacking_actions/90', 
             y_metric='creative_actions/90',
             z_metric='sprints/90',
             title='Presence', 
             show_name_label=True
) 

#df_targets = df_players[mask]

## Excitement

In [55]:
df_players.head(1)

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals/90,minutes/goal,xg/90,xg/shot,xg_overperformance/90,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance/90,goals_outside_box/90,assists/90,minutes/assist,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions/90,xgc/90,xgc_overperformance/90,non_pen_goal_contributions/90,non_pen_xgc/90,non_pen_xgc_overperformance/90,conversion_%,shots/90,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M,non_pen_goals/non_pen_xg/90
0,168222,Mats Seuntjens,"M (LC), AM (RLC), ST (C)",31,187 cm,78 kg,Unh,RKC Waalwijk,Eredivisie,NED,-,-,Scouting Required,"$3,200 p/w",$160K - $1.6M,-,Right,15,1305,6.6,0,87.0,0,0.07,1305.0,0.07,0.06,0.0,0.07,0.07,1305.0,0.07,0.0,0.06,0.0,0.0,0.14,652.5,0.08,0.79,0.05,0.0,0.21,0.15,0.06,0.21,0.15,0.06,7,1.03,0.48,47,0.48,31.45,25.45,81,1.03,3.29,0.9,2.85,0.62,1.97,0.07,0.07,9.38,1.59,16.91,5.38,11.52,-6.14,-53.3,2.0,8.69,4.0,46.03,4.69,53.97,0.0,8.69,52.2,0,0,12.07,2.52,1.03,8.99,0,4.55,18,100,65,57,51,7,5,20,25,16,13,6,51,59,54,1.056,1.0


In [56]:
# Full universe 
scatter_plot(df_targets,
             x_metric='excitement_factor/90', 
             y_metric='sprints/90',
             z_metric='shots_on_target_%',
             title='Distance', 
             show_name_label=True
) 

In [57]:
df_targets.groupby('division')['id'].count()

division
3F Superliga                            7
Bundesliga                              1
Campeonato Brasileiro Série A Assaí    14
Eredivisie                              8
LaLiga EA Sports                        2
Liga Portugal Betclic                   2
Liga Profesional de Fútbol             15
Ligue 1 Uber Eats                       2
Premier League                          2
Serie A TIM                             1
Sky Bet Championship                   13
Name: id, dtype: int64

In [65]:
df_targets.groupby('division').get_group('Sky Bet Championship').sort_values('age')

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals/90,minutes/goal,xg/90,xg/shot,xg_overperformance/90,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance/90,goals_outside_box/90,assists/90,minutes/assist,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions/90,xgc/90,xgc_overperformance/90,non_pen_goal_contributions/90,non_pen_xgc/90,non_pen_xgc_overperformance/90,conversion_%,shots/90,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M,non_pen_goals/non_pen_xg/90
477,2000205927,Jobe Bellingham,"DM, M/AM (C), ST (C)",18,188 cm,80 kg,Inj,Sunderland,Sky Bet Championship,ENG (IRL),Trained in nation (15-21),-,Scouting Required,"$9,250 p/w",$8.4M - $11M,-,Right,19,1381,7.00,1,69.05,477,0.26,345.25,0.10,0.08,0.16,0.26,0.21,345.25,0.10,0.16,0.08,0.16,0.13,0.07,1381.00,0.11,-0.75,-0.05,0.17,0.33,0.22,0.11,0.33,0.22,0.11,21,1.24,0.59,47,0.91,54.87,47.44,86,3.06,5.58,1.37,2.49,1.37,2.49,0.42,0.39,12.84,3.39,26.40,8.80,11.40,-2.60,-22.81,1.04,5.67,3.39,59.77,2.28,40.23,0.98,5.67,69.57,1,0,7.76,2.29,2.02,19.39,0,4.95,54,100,99,68,43,44,30,35,41,47,36,32,90,93,41,11.640,2.600000
473,2000155561,Aidomo Emakhu,"M (R), AM (RL), ST (C)",20,177 cm,68 kg,-,Millwall,Sky Bet Championship,IRL (NGA),-,-,Scouting Required,"$14,750 p/w",$3.9M - $8M,-,Right,8,862,7.02,10,47.89,473,0.73,123.14,0.24,0.14,0.49,0.73,0.41,123.14,0.24,0.49,0.14,0.49,0.21,0.10,862.00,0.15,-0.41,-0.04,0.00,0.84,0.39,0.45,0.84,0.39,0.45,41,1.77,1.25,71,0.73,31.64,26.00,82,2.09,6.60,0.94,2.97,0.94,2.97,0.47,0.42,9.81,2.40,24.47,5.32,11.59,-6.27,-54.10,2.40,10.44,3.86,37.00,6.58,63.00,0.00,10.44,43.33,2,0,10.34,4.68,1.77,9.25,0,7.41,44,100,42,63,73,70,81,67,88,89,88,82,56,56,51,7.140,3.041667
201,28116394,Ellis Simms,ST (C),22,183 cm,81 kg,-,Coventry,Sky Bet Championship,POL (ENG),Trained in nation (15-21),-,Scouting Required,"$18,750 p/w",$6.4M - $10M,-,Right,10,1016,6.89,12,46.18,201,0.53,169.33,0.47,0.16,0.06,0.44,0.15,203.20,0.40,0.04,0.14,0.04,0.00,0.00,0.00,0.09,-1.07,-0.09,0.00,0.53,0.57,-0.03,0.44,0.50,-0.05,18,2.92,1.33,45,0.80,29.41,23.12,79,1.06,3.61,0.89,3.01,0.89,3.01,1.21,0.35,10.01,2.48,24.78,5.40,8.86,-3.46,-39.05,0.89,12.84,6.91,53.79,5.94,46.21,0.09,12.84,57.31,0,0,7.44,2.91,1.33,13.07,0,5.76,23,100,70,21,32,49,42,19,39,46,56,77,86,92,16,9.840,1.100000
198,28108499,Tyrese Campbell,"AM (R), ST (C)",23,183 cm,79 kg,-,Stoke,Sky Bet Championship,ENG,Trained in nation (15-21),-,Scouting Required,"$24,500 p/w",$6.6M - $8.6M,-,Left,21,1722,6.98,4,68.88,198,0.52,172.20,0.44,0.16,0.08,0.52,0.19,172.20,0.44,0.08,0.16,0.08,0.05,0.16,574.00,0.14,0.35,0.02,0.38,0.68,0.58,0.10,0.68,0.58,0.10,19,2.72,0.99,37,1.31,21.64,18.29,85,0.89,4.11,1.05,4.83,0.89,4.11,0.71,0.42,7.06,1.41,20.00,4.97,7.26,-2.29,-31.54,1.31,11.50,3.71,32.27,7.79,67.73,0.16,11.50,36.21,2,0,7.11,3.22,